In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import tushare as ts
# import QUANTAXIS as QA
import talib as ta
import datetime, time
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from tqdm import tqdm_notebook

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

crsp fiscal resample

In [4]:
def calc_exf(df):
    ret = df.ret_p1
    dret = df.dret_p1
    exf = df.mv - df.mv.shift(1) * df.retx_p1
    exf_to_mv = exf / df.mv.shift(1)
    
    car1 = ((ret[::-1]).rolling(1, min_periods=1).apply(np.prod, raw=True)[::-1]) - \
        ((dret[::-1]).rolling(1, min_periods=1).apply(np.prod, raw=True)[::-1])
    car3 = ((ret[::-1]).rolling(3, min_periods=1).apply(np.prod, raw=True)[::-1]) - \
        ((dret[::-1]).rolling(3, min_periods=1).apply(np.prod, raw=True)[::-1])
    car5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1]) - \
        ((dret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])
    
    wr5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1]) / \
        ((dret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])
    
#     ret_n1 = ((ret[::-1]).rolling(1, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n2 = ((ret[::-1]).rolling(2, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n3 = ((ret[::-1]).rolling(3, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n4 = ((ret[::-1]).rolling(4, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n6 = ((ret[::-1]).rolling(6, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n7 = ((ret[::-1]).rolling(7, min_periods=1).apply(np.prod, raw=True)[::-1])
#     ret_n8 = ((ret[::-1]).rolling(8, min_periods=1).apply(np.prod, raw=True)[::-1])
    
    res = pd.DataFrame({
        'date':df.date,
        'mv':df.mv,
        'retx_p1':df.retx_p1,
        'exf':exf,
        'exf_to_mv':exf_to_mv,
        'car1':car1.shift(-1),
        'car3':car3.shift(-1),
        'car5':car5.shift(-1),
        'wr5':wr5.shift(-1),
#         'ret_n1':ret_n1.shift(-1),
#         'ret_n2':ret_n2.shift(-1),
#         'ret_n3':ret_n3.shift(-1),
#         'ret_n4':ret_n4.shift(-1),
#         'ret_n5':ret_n5.shift(-1),
#         'ret_n6':ret_n6.shift(-1),
#         'ret_n7':ret_n7.shift(-1),
#         'ret_n8':ret_n8.shift(-1)
    })
    return res

In [5]:
crsp_fa = pd.read_csv('data/crsp_fa_filtered.zip',
                     parse_dates=['date'], infer_datetime_format=True)

In [6]:
crsp_fa.head()

,date,permno,ret_p1,retx_p1,dret_p1,mv,mv_adj
0,1986-10-31,10000,0.178571,0.178571,1.059979,3002.34375,5796.726244
1,1986-06-30,10001,1.032450,1.000000,1.252963,6033.12500,12067.131553
2,1987-06-30,10001,1.023885,0.959184,1.018842,5822.12500,11205.083735
3,1988-06-30,10001,1.140535,1.063829,0.827947,6200.00000,11459.064979
4,1989-06-30,10001,1.199811,1.120000,1.005611,7007.00000,12326.466763


In [16]:
crsp_fa.query('permno == 77418')

,date,permno,ret_p1,retx_p1,dret_p1,mv,mv_adj
149844,1992-06-30,77418,0.885246,0.885246,0.902617,7.399350e+04,1.153435e+05
149845,1993-06-30,77418,2.741783,2.740738,1.256202,2.180040e+05,3.304247e+05
149846,1994-06-30,77418,1.540542,1.540542,1.046047,4.129650e+05,6.094740e+05
149847,1995-06-30,77418,3.087717,3.087717,1.174147,1.652420e+06,2.371602e+06
149848,1996-06-30,77418,1.988631,1.988631,1.220468,3.935138e+06,5.489542e+06
149849,1997-06-30,77418,1.271430,1.271430,1.321836,5.443073e+06,7.432170e+06
149850,1998-06-30,77418,3.823007,3.822978,1.302787,2.272824e+07,3.050151e+07
149851,1999-06-30,77418,4.185493,4.185493,1.216303,1.195158e+08,1.570238e+08
149852,2000-06-30,77418,0.957955,0.957955,1.091818,1.216507e+08,1.542759e+08
149853,2001-12-31,77418,0.922413,0.922413,0.863769,1.365999e+08,1.631004e+08


In [20]:
test = crsp_fa.query('permno == 77418').set_index('date')

In [24]:
test['2002':'2006']

,permno,ret_p1,retx_p1,dret_p1,mv,mv_adj
date,,,,,,
2002-12-31,77418,0.408099,0.408099,0.784459,56316389.10,6.543742e+07
2003-12-31,77418,1.373283,1.373283,1.299333,78266574.40,8.913651e+07
2004-12-31,77418,1.081156,1.081156,1.116584,85758706.60,9.496747e+07
2005-12-31,77418,0.901661,0.896659,1.072051,79794365.60,8.494877e+07
2006-12-31,77418,1.263697,1.248854,1.160715,86522639.94,9.023852e+07


In [26]:
test['2002':'2006'].ret_p1

date
2002-12-31    0.408099
2003-12-31    1.373283
2004-12-31    1.081156
2005-12-31    0.901661
2006-12-31    1.263697
Name: ret_p1, dtype: float64

In [28]:
p = 1
for i in test['2002':'2006'].ret_p1:
    p = i * p
print(p)

0.6904002043681601


In [27]:
test['2002':'2006'].ret_p1.prod()

0.69040020436816008

In [29]:
test['2002':'2006'].dret_p1.prod()

1.4161960357477106

In [31]:
wr5_test = test['2002':'2006'].ret_p1.prod() / test['2002':'2006'].dret_p1.prod()

In [32]:
wr5_test

0.48750327422265993

In [ ]:
wr5 = ((ret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1]) / \
        ((dret[::-1]).rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1])

In [34]:
s = pd.Series(range(1,11))

In [52]:
s

0     1
1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9    10
dtype: int64

In [53]:
s[::-1]

9    10
8     9
7     8
6     7
5     6
4     5
3     4
2     3
1     2
0     1
dtype: int64

In [35]:
s.rolling(5).sum()

0     NaN
1     NaN
2     NaN
3     NaN
4    15.0
5    20.0
6    25.0
7    30.0
8    35.0
9    40.0
dtype: float64

In [54]:
s[::-1].rolling(5).sum()

9     NaN
8     NaN
7     NaN
6     NaN
5    40.0
4    35.0
3    30.0
2    25.0
1    20.0
0    15.0
dtype: float64

In [55]:
s[::-1].rolling(5).sum()[::-1]

0    15.0
1    20.0
2    25.0
3    30.0
4    35.0
5    40.0
6     NaN
7     NaN
8     NaN
9     NaN
dtype: float64

In [56]:
s[::-1].rolling(5).sum()[::-1].shift(-1)

0    20.0
1    25.0
2    30.0
3    35.0
4    40.0
5     NaN
6     NaN
7     NaN
8     NaN
9     NaN
dtype: float64

In [58]:
s[::-1].rolling(5, min_periods=1).sum()[::-1].shift(-1)

0    20.0
1    25.0
2    30.0
3    35.0
4    40.0
5    34.0
6    27.0
7    19.0
8    10.0
9     NaN
dtype: float64

In [50]:
s.rolling(5, min_periods=1).sum()

0     1.0
1     3.0
2     6.0
3    10.0
4    15.0
5    20.0
6    25.0
7    30.0
8    35.0
9    40.0
dtype: float64

In [49]:
s.rolling(5, min_periods=1).apply(np.prod, raw=True)

0        1.0
1        2.0
2        6.0
3       24.0
4      120.0
5      720.0
6     2520.0
7     6720.0
8    15120.0
9    30240.0
dtype: float64

In [43]:
test.ret_p1

date
1992-06-30    0.885246
1993-06-30    2.741783
1994-06-30    1.540542
1995-06-30    3.087717
1996-06-30    1.988631
1997-06-30    1.271430
1998-06-30    3.823007
1999-06-30    4.185493
2000-06-30    0.957955
2001-12-31    0.922413
2002-12-31    0.408099
2003-12-31    1.373283
2004-12-31    1.081156
2005-12-31    0.901661
2006-12-31    1.263697
2007-12-31    0.767318
2008-12-31    0.620778
2009-12-31    1.341498
2010-12-31    1.135748
2011-12-31    1.153788
2012-12-31    1.358594
2013-12-31    1.486231
Name: ret_p1, dtype: float64

In [59]:
test.ret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1)
# test.ret_p1.rolling(5, min_periods=1).apply(np.prod, raw=True)

date
1992-06-30     32.975429
1993-06-30     45.979314
1994-06-30    124.921070
1995-06-30     38.756382
1996-06-30     17.976892
1997-06-30      5.770165
1998-06-30      2.072731
1999-06-30      0.535408
2000-06-30      0.503945
2001-12-31      0.690400
2002-12-31      1.298106
2003-12-31      0.586795
2004-12-31      0.728095
2005-12-31      0.917121
2006-12-31      0.837355
2007-12-31      1.482600
2008-12-31      3.549555
2009-12-31      2.645964
2010-12-31      2.329711
2011-12-31      2.019184
2012-12-31      1.486231
2013-12-31           NaN
Name: ret_p1, dtype: float64

In [60]:
test.dret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1)

date
1992-06-30    2.489071
1993-06-30    2.581374
1994-06-30    3.001522
1995-06-30    2.791061
1996-06-30    1.975333
1997-06-30    1.172284
1998-06-30    1.169177
1999-06-30    1.073322
2000-06-30    1.053890
2001-12-31    1.416196
2002-12-31    1.955406
2003-12-31    0.931225
2004-12-31    1.070250
2005-12-31    1.155372
2006-12-31    0.992235
2007-12-31    1.061177
2008-12-31    2.237557
2009-12-31    1.743622
2010-12-31    1.506608
2011-12-31    1.511410
2012-12-31    1.304742
2013-12-31         NaN
Name: dret_p1, dtype: float64

In [61]:
(test.ret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1)) / \
(test.dret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1))

date
1992-06-30    13.248084
1993-06-30    17.811951
1994-06-30    41.619246
1995-06-30    13.885897
1996-06-30     9.100687
1997-06-30     4.922155
1998-06-30     1.772812
1999-06-30     0.498832
2000-06-30     0.478177
2001-12-31     0.487503
2002-12-31     0.663855
2003-12-31     0.630132
2004-12-31     0.680304
2005-12-31     0.793789
2006-12-31     0.843908
2007-12-31     1.397127
2008-12-31     1.586353
2009-12-31     1.517510
2010-12-31     1.546328
2011-12-31     1.335961
2012-12-31     1.139100
2013-12-31          NaN
dtype: float64

In [62]:
(test.ret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1)) - \
(test.dret_p1[::-1].rolling(5, min_periods=1).apply(np.prod, raw=True)[::-1].shift(-1))

date
1992-06-30     30.486357
1993-06-30     43.397940
1994-06-30    121.919548
1995-06-30     35.965321
1996-06-30     16.001558
1997-06-30      4.597881
1998-06-30      0.903554
1999-06-30     -0.537914
2000-06-30     -0.549944
2001-12-31     -0.725796
2002-12-31     -0.657300
2003-12-31     -0.344430
2004-12-31     -0.342154
2005-12-31     -0.238251
2006-12-31     -0.154880
2007-12-31      0.421423
2008-12-31      1.311998
2009-12-31      0.902342
2010-12-31      0.823103
2011-12-31      0.507775
2012-12-31      0.181489
2013-12-31           NaN
dtype: float64

In [6]:
crsp_fa.head()

,date,permno,ret_p1,retx_p1,dret_p1,mv,mv_adj
0,1986-10-31,10000,0.178571,0.178571,1.059979,3002.34375,5796.726244
1,1986-06-30,10001,1.032450,1.000000,1.252963,6033.12500,12067.131553
2,1987-06-30,10001,1.023885,0.959184,1.018842,5822.12500,11205.083735
3,1988-06-30,10001,1.140535,1.063829,0.827947,6200.00000,11459.064979
4,1989-06-30,10001,1.199811,1.120000,1.005611,7007.00000,12326.466763


验证mv adjust是否正确

In [7]:
cpi = pd.read_excel('data/CPIAUCSL.xlsx', index_col=0)

In [11]:
crsp_fa = crsp_fa.set_index('date')
(crsp_fa['1986-10-31'].query('permno == 10000').mv.iloc[0] / \
cpi['1986-11'].cpi_adjust.iloc[0]) - \
(crsp_fa['1986-10-31'].query('permno == 10000').mv_adj.iloc[0])

9.0949470177292824e-13

In [12]:
crsp_fa = crsp_fa.reset_index()

查看firm-years数量

ps:因为流通股数的单位默认是1000, 故mv_adj只需要大于50000即可

In [8]:
# crsp_fa[crsp_fa.mv_adj >= 5e+4].shape[0]

145087

In [7]:
crsp_fa = crsp_fa.reset_index().set_index('date')[:'2008']
crsp_fa[(crsp_fa.mv_adj.shift(1) * crsp_fa.retx_p1) >= 5e+4].shape[0]

140820

In [8]:
# crsp_fa = crsp_fa[crsp_fa.mv_adj >= 5e+4]
crsp_fa = crsp_fa[crsp_fa.mv_adj.shift(1) * crsp_fa.retx_p1 >= 5e+4]

In [9]:
crsp_fa = crsp_fa.reset_index()

In [10]:
crsp_fa.head()

,date,index,permno,ret_p1,retx_p1,dret_p1,mv,mv_adj
0,1994-12-31,37,10002,1.039473,1.019418,0.927850,39361.875,55492.135988
1,1995-12-31,38,10002,1.092129,1.066668,1.373491,42028.000,57642.203439
2,1996-12-31,39,10002,0.955922,0.928571,1.116432,39026.000,51946.690866
3,1997-12-31,40,10002,1.962960,1.884614,1.257241,104027.000,136245.831469
4,1998-12-31,41,10002,0.951829,0.933673,0.942657,117867.250,151841.930185


In [11]:
tic = time.perf_counter()
crsp_fa = crsp_fa.set_index('permno').groupby('permno').apply(calc_exf)
toc = time.perf_counter()
print(toc-tic)

157.3110456



exf_to_mv : $EXF_{t}/MV_{t-1}$

ret_n1 : 接下来1年的return


In [14]:
from scipy.stats import mstats

In [12]:
crsp_fa.head()

,date,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5
permno,,,,,,,,,
10002,1994-12-31,39361.875,1.019418,NaN,NaN,-0.281362,0.121447,-0.897720,0.640625
10002,1995-12-31,42028.000,1.066668,41.937460,1.065433e-03,-0.160509,0.462912,-0.522323,0.688416
10002,1996-12-31,39026.000,0.928571,-0.001807,-4.298500e-08,0.705720,-0.096200,-0.527141,0.719418
10002,1997-12-31,104027.000,1.884614,30478.045526,7.809677e-01,0.009172,-0.579293,-0.533391,0.601983
10002,1998-12-31,117867.250,0.933673,20740.033634,1.993716e-01,-0.554149,-0.861840,-1.102672,0.534312


winsorize

In [26]:
crsp_fa1 = crsp_fa.copy()
crsp_fa1 = crsp_fa1.dropna(subset=['exf_to_mv'])
crsp_fa1 = crsp_fa1.reset_index().set_index('date')
# crsp_fa1 = crsp_fa1.loc['1973':'2001']
# 可以自行调整limits大小
crsp_fa1.exf_to_mv = mstats.winsorize(crsp_fa1.exf_to_mv, limits=[0.01, 0.01])

per1 = np.linspace(0,1,11).tolist()
bins = crsp_fa1.describe(percentiles=per1).exf_to_mv.iloc[4:-1].tolist()
bins = [bins[0]-1] + bins[1:]
# bins = bins[:-1] + [bins[-1]-1]
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).mean().drop(['mv', 'retx_p1', 'exf'], axis=1)

,permno,exf_to_mv,car1,car3,car5,wr5
exf_to_mv,,,,,,
"(-2.008, -0.0433]",57375.578350,-0.280491,0.153113,0.359148,0.549825,1.329062
"(-0.0433, -0.00669]",56128.387450,-0.021075,0.049655,0.157581,0.253104,1.170257
"(-0.00669, -1.33e-07]",55818.941662,-0.001465,0.045738,0.148779,0.250850,1.160143
"(-1.33e-07, 0.000943]",54902.980472,0.000192,0.056655,0.161470,0.272402,1.169021
"(0.000943, 0.00465]",54940.159817,0.002642,0.066227,0.155971,0.246640,1.150438
"(0.00465, 0.0111]",55714.778985,0.007550,0.060459,0.142160,0.224420,1.135947
"(0.0111, 0.0242]",56807.320042,0.016756,0.048486,0.150280,0.241860,1.147183
"(0.0242, 0.0615]",58420.502982,0.039006,0.053467,0.147126,0.194807,1.118683
"(0.0615, 0.195]",58847.675954,0.114021,0.021106,0.074663,0.105116,1.069604


In [23]:
crsp_fa.exf_to_mv.shape[0]

140820

In [24]:
crsp_fa.exf_to_mv.dropna().shape[0]

122392

Significant issuance & Significant repurchase

EXF is greater than 5% of pre-issuance market capitalization

$EXF_t >= 0.05 * (1+rx_t) *MV_{t-1}$

equals

$EXF_t / MV_{t-1} >= 0.05 * (1+rx_t)$

In [28]:
crsp_fa1[crsp_fa1.exf_to_mv > (0.05 * crsp_fa1.retx_p1)].describe().loc[['count', 'mean']]

,permno,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5
count,25774.000000,2.577400e+04,25774.000000,25774.000000,25774.000000,22915.000000,22915.000000,22915.000000,22915.000000
mean,60756.553154,1.474336e+06,1.304114,282855.944404,0.361538,0.022519,0.051094,0.095825,1.056491


In [29]:
crsp_fa1[crsp_fa1.exf_to_mv <= (-0.05 * crsp_fa1.retx_p1)].describe().loc[['count', 'mean']]

,permno,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5
count,10988.000000,1.098800e+04,10988.000000,10988.000000,10988.000000,9113.000000,9113.000000,9113.000000,9113.000000
mean,58098.366127,1.656792e+06,1.477054,-258574.958229,-0.303696,0.169161,0.399301,0.600632,1.355417


In [30]:
crsp_fa1[(crsp_fa1.exf_to_mv >(-0.05 * crsp_fa1.retx_p1)) & \
         (crsp_fa1.exf_to_mv <= (0.05 * crsp_fa1.retx_p1))].describe().loc[['count', 'mean']]

,permno,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5
count,85630.000000,8.563000e+04,85630.000000,85630.00000,85630.00000,76169.000000,76169.000000,76169.000000,76169.000000
mean,55858.441037,1.997534e+06,1.122395,-3343.08724,0.00523,0.052773,0.147237,0.235811,1.148383


long Significant repurchase,  short Significant issuance

In [32]:
crsp_fa1[crsp_fa1.exf_to_mv <= (-0.05 * crsp_fa1.retx_p1)].describe().loc[['mean']] - \
crsp_fa1[crsp_fa1.exf_to_mv > (0.05 * crsp_fa1.retx_p1)].describe().loc[['mean']]

,mv,retx_p1,exf,exf_to_mv,car1,car3,car5,wr5,ret_n1,ret_n2,ret_n3,ret_n4,ret_n5,ret_n6,ret_n7,ret_n8
mean,-367175.786559,0.32726,-578064.095035,-0.844436,0.234454,0.611467,1.180597,0.574668,0.264126,0.444009,0.671183,0.936371,1.26131,1.549077,1.670117,1.760836


查看样本的个数

In [27]:
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).count().exf_to_mv

exf_to_mv
(-2.008, -0.0433]        12240
(-0.0433, -0.00669]      12239
(-0.00669, -1.33e-07]    12239
(-1.33e-07, 0.000943]    12239
(0.000943, 0.00465]      12239
(0.00465, 0.0111]        12239
(0.0111, 0.0242]         12239
(0.0242, 0.0615]         12239
(0.0615, 0.195]          12239
(0.195, 1.841]           12240
Name: exf_to_mv, dtype: int64

In [163]:
result = crsp_fa.copy()

In [167]:
writer = pd.ExcelWriter('data/sample_fiscal.xlsx', datetime_format='YYYY-MM-DD')
crsp_fa[crsp_fa.permno==77418].to_excel(writer, index=False)
writer.save()

annual sorts

In [ ]:
crsp_fa1 = crsp_fa.copy()
crsp_fa1 = crsp_fa1.dropna(subset=['exf_to_mv'])
crsp_fa1 = crsp_fa1.reset_index().set_index('date')
# crsp_fa1 = crsp_fa1.loc['1973':'2001']
# 可以自行调整limits大小
crsp_fa1.exf_to_mv = mstats.winsorize(crsp_fa1.exf_to_mv, limits=[0.01, 0.01])

per1 = np.linspace(0,1,11).tolist()
bins = crsp_fa1.describe(percentiles=per1).exf_to_mv.iloc[4:-1].tolist()
bins = [bins[0]-1] + bins[1:]
# bins = bins[:-1] + [bins[-1]-1]
crsp_fa1.groupby(pd.cut(crsp_fa1.exf_to_mv,bins, right=True)).mean().drop(['mv', 'retx_p1', 'exf'], axis=1)

In [31]:
crsp_fa2 = crsp_fa.copy()
crsp_fa2 = crsp_fa2.dropna(subset=['exf_to_mv'])
crsp_fa2 = crsp_fa2.reset_index().set_index('date')

In [39]:
def group_decile(df):
#     df.exf_to_mv = mstats.winsorize(df.exf_to_mv, limits=[0.01, 0.01])
#     per1 = np.linspace(0,1,11).tolist()
#     bins = df.describe(percentiles=per1).exf_to_mv.iloc[4:-1].tolist()
#     print(bins)
#     bins = [bins[0]-1] + bins[1:]
    res = df.groupby(pd.cut(df.exf_to_mv,10, right=True)).mean().drop(['mv', 'retx_p1', 'exf'], axis=1)
    return res

In [40]:
test = crsp_fa2.groupby('date').apply(group_decile)

TypeError: categories must match existing categories when appending